# 줄리아를 생각하다(Think julia)

## Chapter 20. 알아두면 좋은 것들: Base 및 표준 라이브러리

* 줄리아에는 기능 세트가 기본으로 포함되어 있음

* Base 모듈에는 가장 유용한 함수, 자료형, 매크로가 있음

* 줄리아의 표준 라이브러리에는 많은 종류의 전문적 모듈이 포함되어 있음

    - 날짜 처리, 분산 컴퓨팅, 선형대수, 성능 측정, 랜덤 숫자 등에 관한 모듈이 있음
```
```
    - 표준 라이브러리에 있는 함수, 자료형, 매크로는 사용하기 전에 코드로 반입(import)되어야 함
```
```
        - **import Module** 문은 모듈을 반입. Module.fn(x)로 함수 fn을 호출할 수 있음
```
```
        - **using Module** 문은 그 모듈의 모든 함수, 자료형, 매크로를 반입

* ```import``` 와 ```using```의 차이점

    * **import**는 모듈을 메모리 상에 적재하는 역할을 하고, **using**은 적재 뿐만 아니라 모듈에서 export하는 객체명들을 직접적으로 사용할 수 있음 

    * 각 모듈의 객체를 global namespace에 적재해서 객체명 그대로 사용하고자 한다면 ```using```을 이용하는 것이 효율적임
    
    * _Names referring to functions, types, global variables, and constants_


**(참고)** https://docs.julialang.org/en/v1/manual/modules/#modules

    * Importantly, using ModuleName is the only form for which export lists matter at all.

    * Usually, import ModuleName is used in contexts when the user wants to keep the namespace clean.




_```import Foo``` will load the module or package Foo. Names from the imported Foo module can be accessed with dot syntax (e.g. Foo.foo to access the name foo). See the manual section about modules for details._
        
_```using Foo``` will load the module or package Foo and make its exported names available for direct use. Names can also be used via dot syntax (e.g. Foo.foo to access the name foo), whether they are exported or not. See the manual section about modules for details._


![image](capture_and_drawing/Think_julia_drawing3.png)

### 20.1 성능 측정

* **BenchmarkTools** 패키지에서 제공하는 ```@time``` 매크로를 활용하면 수행 시간, 할당 횟수, 할당된 메모리를 출력함

    * python의 %timeit 또는 %time과 유사

In [1]:
function fib(n)
    if n == 0
        return 0
    elseif n == 1
        return 1
    else
        return fib(n-1) + fib(n-2)
    end
end

known = Dict(0 => 0, 1 => 1) # 메모장 역할을 하는 변수 known 생성

function fibonacci(n)
    if n ∈keys(known)
        return known[n]
    end
    res = fibonacci(n-1) + fibonacci(n-2)
    known[n] = res # 새로운 값은 메모장에 기록
    res
end

fibonacci (generic function with 1 method)

In [2]:
fib(1)

1

In [3]:
fibonacci(1)

1

In [4]:
using BenchmarkTools

In [9]:
@time fib(40) # 2번째 실행 시간을 체크해야 함

  0.497073 seconds


102334155

In [10]:
@time fibonacci(40) # 2번째 실행 시간을 체크해야 함

  0.000008 seconds (2 allocations: 32 bytes)


102334155

In [11]:
0.497073/0.000008  # 약 6만배 빨라짐

62134.125

메모이제이션을 사용한 fibonacci() 함수의 메모리 사용량이 많은 것을 확인할 수 있음

**(주의)** 줄리아의 함수는 처음 실행 시 컴파일 됨. 따라서 두 개의 알고리즘을 비교하려면, 알고리즘을 함수로 구현해서 컴파일될 수 있도록 첫 번째 실행의 성능 수치는 제외해야 함. 그렇지 않으면 컴파일 시간이 측정 수치에 포함됨 

-> 이런 요소까지 고려해 성능 측정을 해주는 ```@btime``` 매크로 사용 가능

In [12]:
@btime fib(40)

  512.336 ms (0 allocations: 0 bytes)


102334155

In [13]:
@btime fibonacci(40)

  51.111 ns (2 allocations: 32 bytes)


102334155

In [15]:
512.336 * 10^6 / 51.111 # 약 천만배 (10^7) 빨라짐

1.0023987008667411e7

### 20.2 컬렉션과 자료구조

* **집합(set)** : 값 없이 사전의 키만 모아놓은 것처럼 동작하는 자료형태

    - 집합에 원소를 추가하거나 원소가 있는지 확인하는 것은 빠르게 동작함 
    
    - 일반적인 집합 연산에 사용하는 연산자와 함수도 제공
```
```
* **예시)** ```setdiff```를 이용하여 차집합 계산

In [16]:
function subtract(d1, d2)
    res = Dict()
    for key in keys(d1)
        if key ∉ keys(d2)
            res[key] = nothing 
        end
    end
    res
end

subtract (generic function with 1 method)

In [17]:
function subtract2(d1, d2)
    setdiff(d1, d2)
end

subtract2 (generic function with 1 method)

* **예시2)** 집합을 이용하여 간결하고 효율적으로 함수 구현

In [19]:
# 원소가 처음으로 나타나면 딕셔너리에 추가하고 같은 원소가 또 나타나면 true를 반환하는 함수

function hasduplicates(t)
    d = Dict()
    for x in t
        if x ∈ d
            return true
        end
        d[x] = nothing
    end
    false
end

hasduplicates (generic function with 1 method)

In [20]:
# 집합에서 원소는 딱 한 번 나오므로, t에 두 번 이상 나오는 원소가 있다면 집합은 t보다 작을 것임

# 중복이 없다면 집합의 크기는 t와 같을 것임

function hasduplicates2(t)
    length(Set(t)) < length(t)
end

hasduplicates2 (generic function with 1 method)

In [21]:
# word의 글자가 available에 있는지 확인하는 useonly 함수 

function useonly(word, available)
    for letter in word
        if letter ∉ available
            return false
        end
    end
    true
end

useonly (generic function with 1 method)

In [22]:
# 집합을 사용하여 수정한 useonly 함수

function useonly2(word, available)
    Set(word) ⊆ Set(available)
end

useonly2 (generic function with 1 method)

* _⊆(\subseteq + tab)은 한 집합이 다른 집합의 부분집합인지를 확인하는 연산자_

### 20.3 수학

* 줄리아는 복소수도 지원함

In [23]:
ℯ^(im*π)+1

0.0 + 1.2246467991473532e-16im

In [24]:
x = 0:0.1:2π

cos.(x) == 0.5*(ℯ.^(im*x) + ℯ.^(-im * x))

true

### 20.4 문자열

* 줄리아는 perl과 호환되는 정규표현식(regular expression)도 사용 가능

* 정규표현식은 보통 regex라고 하는데 문자열에서 복잡한 패턴을 쉽게 찾을 수 있게 해줌

* (참조) https://docs.julialang.org/en/v1/manual/strings/#man-regex-literals

In [29]:
function useonly3(word, available)
    r = Regex("[^$(available)]") # available 문자열에 나오지 않는 글자를 찾는 패턴
    !occursin(r, word) # 그 패턴을 word에서 찾으면 true 반환, ! 붙었으니까 문자열에 나오지 않는 패턴이 없으면 true (모두 존재한다는 의미)
end

useonly3 (generic function with 1 method)

In [26]:
useonly3("bananan", "abn")

true

In [27]:
useonly3("bananas", "abn")

false

In [30]:
# 정규표현식은 r을 접두어로 붙인 비표준 문자열을 사용해 만들 수 있음

match(r"[^abn]", "banana")
m = match(r"[^abn]", "bananas")

RegexMatch("s")

In [32]:
println("m.match: ", m.match, "\n",
        "m.offset: ", m.offset)

m.match: s
m.offset: 7


* match 함수는 패턴이 발견되면 RegexMatch 개체를 반환하고, 그렇지 않으면 nothing을 반환함

* 반환된 RegexMatch 객체에서 다음과 같은 정보를 알아낼 수 있음

    - m.match : 패턴과 일치하는 부분 문자열 전체
    
    - m.capture : 캡처된 부분 문자열의 배열
    
    - m.offset : 패턴과 일치하는 부문 문자열 전체의 시작점
    
    - m.offsets : 캡처된 부분 문자열들의 위치 배열
    

### 20.5 배열

* **행렬(matrix)** : 2차원 배열

* 2 x 3 행렬 만들기

In [33]:
z = zeros(Float64, 2, 3)

2×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0

In [34]:
size(z)

(2, 3)

In [35]:
s = ones(String, 1, 3)

1×3 Matrix{String}:
 ""  ""  ""

* 배열은 빈칸으로 한 행의 원소를 구분하고, 세미콜론(;)으로 행을 구분하는 방식으로 직접 입력할 수 있음

In [36]:
a = [1 2 3 ; 4 5 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

* 개별 원소에 접근하기 위해 대괄호를 사용할 수 있음

In [37]:
z[1, 2] = 1; 
z[2, 3] = 1;
z

2×3 Matrix{Float64}:
 0.0  1.0  0.0
 0.0  0.0  1.0

* 문자열처럼 조각(slice)를 이용해 부분 행렬을 가져올 수 있음

In [38]:
u = z[:, 2:end]

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

 * 도트 연산자는 모든 차원의 원소에 다 적용됨

In [39]:
ℯ.^(im * u)

2×2 Matrix{ComplexF64}:
 0.540302+0.841471im       1.0+0.0im
      1.0+0.0im       0.540302+0.841471im

### 20.6 인터페이스

* 줄리아는 어떤 동작을 정의하기 위해 몇 가지 비격식 인터페이스, 즉 특정 목적의 메서드를 사용함

* 그런 메서드를 확장해 어떤 자료형에 대응할 수 있게 하면, 그 자료형의 객체도 그 동작의 대상이 될 수 있음

* 어떤 컬렉션의 값들, 혹은 반복에 대해 루프를 도는 것도 인터페이스에 해당

* **예시1)** 피보나치 수열을 느긋하게 반환하는 반복자 만들기 

In [40]:
struct Fibonacci{T<:Real} end
Fibonacci(d::DataType) = d<:Real ? Fibonacci{d}() : error("No Real type!")

Base.iterate(::Fibonacci{T}) where {T<:Real} = (zero(T), (one(T), one(T)))
Base.iterate(::Fibonacci{T}, state::Tuple{T, T}) where {T<:Real} = (state[1], (state[2], state[1] + state[2]))

In [41]:
for e in Fibonacci(Int64)
    e > 100 && break
    print(e, " ")
end

0 1 1 2 3 5 8 13 21 34 55 89 

내부적으로 for 루프는 아래와 같이 변환되어 실제 그 값이 꼭 필요해지기 전까지는 연산하지 않고는 lazy 연산을 수행함

---
```julia
for i in iter
    # body
end
```
---

```julia

next = iterate(iter)
while next !== nothing
    (i, state) = next
    # body
    next = iterate(iter, state)
end

```

### 20.7 대화형 도구

* LLVM 라이브러리는 줄리아 코드를 CPU가 실행할 수 있는 기계(machine code)로 번역하는 과정을 여러 단계를 거쳐 수행함

* ```using InteractiveUtilities``` (모듈)을 이용하여 각 단계의 출력을 직접적으로 시각화해서 볼 수 있음
```
```
    * **@code_lowered** : 컴파일러가 최적화된 코드를 만들어내기 전 단계로 생성하는 중간 표현(intermediate representation)을 배열로 반환하며 중간 표현을 확인할 수 있는 매크로(저수준화 lowered 표현 확인 매크로)
```
```
    * **@code_typed** : 각 단계의 결과와 반환값의 자료형이 잘 맞게 추론되었음을 확인하는 매크로
```
```
    * **@code_llvm** : 중간 변환된 코드는 llvm 코드로 변환, 이때 변환된 llvm 코드를 확인하는 매크로 
```
```
    * **@code_native** : 최종적으로 생성된 기계어를 확인하는 매크로

In [42]:
function squaresum(a::Float64, b::Float64)
    a^2 + b^2
end

squaresum (generic function with 1 method)

In [43]:
using InteractiveUtils

In [44]:
@code_lowered squaresum(3.0, 4.0)

CodeInfo(
1 ─ %1 = Core.apply_type(Base.Val, 2)
│   %2 = (%1)()
│   %3 = Base.literal_pow(Main.:^, a, %2)
│   %4 = Core.apply_type(Base.Val, 2)
│   %5 = (%4)()
│   %6 = Base.literal_pow(Main.:^, b, %5)
│   %7 = %3 + %6
└──      return %7
)

In [45]:
@code_typed squaresum(3.0, 4.0)

CodeInfo(
1 ─ %1 = Base.mul_float(a, a)::Float64
│   %2 = Base.mul_float(b, b)::Float64
│   %3 = Base.add_float(%1, %2)::Float64
└──      return %3
) => Float64

In [46]:
@code_llvm squaresum(3.0, 4.0)

;  @ In[42]:1 within `squaresum`
; Function Attrs: uwtable
define double @julia_squaresum_3477(double %0, double %1) #0 {
top:
;  @ In[42]:2 within `squaresum`
; ┌ @ intfuncs.jl:321 within `literal_pow`
; │┌ @ float.jl:385 within `*`
    %2 = fmul double %0, %0
    %3 = fmul double %1, %1
; └└
; ┌ @ float.jl:383 within `+`
   %4 = fadd double %2, %3
; └
  ret double %4
}


In [47]:
@code_native squaresum(3.0, 4.0)

	.text
	.file	"squaresum"
	.globl	julia_squaresum_3514            # -- Begin function julia_squaresum_3514
	.p2align	4, 0x90
	.type	julia_squaresum_3514,@function
julia_squaresum_3514:                   # @julia_squaresum_3514
; ┌ @ In[42]:1 within `squaresum`
	.cfi_startproc
# %bb.0:                                # %top
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register %rbp
; │ @ In[42]:2 within `squaresum`
; │┌ @ intfuncs.jl:321 within `literal_pow`
; ││┌ @ float.jl:385 within `*`
	vmulsd	%xmm0, %xmm0, %xmm0
	vmulsd	%xmm1, %xmm1, %xmm1
; │└└
; │┌ @ float.jl:383 within `+`
	vaddsd	%xmm1, %xmm0, %xmm0
; │└
	popq	%rbp
	retq
.Lfunc_end0:
	.size	julia_squaresum_3514, .Lfunc_end0-julia_squaresum_3514
	.cfi_endproc
; └
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


### 20.8 디버깅

* Logging 매크로는 print문을 이용한 스캐폴딩에 대한 대안을 제공함

* **@debug** 매크로를 사용했다면, 이를 소스코드에서 제거할 필요는 없음

* **@warn**과 다르게 **@debug**는 기본적으로 아무런 출력을 하지 않음

In [48]:
@warn "Abandon printf debugging, all ye who enter here!"

┌ Warning: Abandon printf debugging, all ye who enter here!
└ @ Main In[48]:1


In [49]:
@debug "The sum of some values $(sum(rand(100)))"

* 디버그 메시지를 로그  파일에 저장하기 위해 **디버그 로킹** (debug logging)을 활성화한 경우가 아니라면 sum(rand(100))은 전혀 평가되지 않음

* 쉘 스크립트에서 로깅의 수준은 환경 변수 JULIA_DEBUG 값으로 조정 가능

* 예시) 환경 변수 JULIA_DEBUG 수정 쉘 스크립트
```sh
$JULIA_DEBUG=all julia -e '@debug "The sum of some values $(sum(rand(100)))"'
```

### 20.9 용어집

* **집합** (set)

    - 구별되는 객체들의 모음
```
```
* **정규표현식** (regular expression)

    - regex라고도 하며 탐색 패턴을 정의한 문자열
    
    - (참고) https://docs.julialang.org/en/v1/manual/strings/#man-regex-literals
```
```
* **행렬** (matrix)

    - 2차원 배열
```
```
* **기계어** (machine code)

    - 컴퓨터의 CPU에서 바로 실행될 수 있는 명령어들
```
```
* **중간 표현** (intermediate representation)

    - 컴파일러가 소스 코드를 다루기 위해 내부적으로 사용하는 자료구조
```
```
* **디버그 로깅** (debug logging)

    - 디버그 메시지를 로그 파일에 저장하는 일